In [131]:
import numpy as np
import torch
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.optim as optim

In [132]:
#transform to 224, 224 since VGG expect that size
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
])

# Define paths to your train and validation data
train_path = "train"
val_path = "val"

#define train and valdataset again but with transoformers this time
train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)

#8 batch size to reduce training time
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#20% of training since otherwise it would have taken a day to train
subset_size = int(0.2 * len(train_dataset))

# Randomly select a subset of indices from the training dataset
subset_indices = np.random.choice(len(train_dataset), subset_size, replace=False)

# Create a subset of the training dataset using the selected indices
train_subset = Subset(train_dataset, subset_indices)

# Use DataLoader to load the subset of data in batches
train_loader_subset = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

# Print dataset details
print("Number of classes:", len(train_dataset.classes))
print("Classes:", train_dataset.classes)
print("Number of training images:", len(train_dataset))
print("Number of training subset images:", len(train_subset))
print("Number of validation images:", len(val_dataset))

Number of classes: 2
Classes: ['MEL', 'NV']
Number of training images: 6426
Number of training subset images: 1285
Number of validation images: 1252


<h2> Some baseline model

In [133]:
import torch.nn as nn
import torch.nn.functional as F

In [134]:
class SimpleCNN(nn.Module):
    
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 28 * 28)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

In [135]:
from tqdm import tqdm  # Import tqdm for progress bar

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the model
num_classes = len(train_dataset.classes)
model = SimpleCNN(num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    # Use tqdm for progress bar
    with tqdm(train_loader_subset, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update tqdm progress bar
            t.set_postfix(loss=running_loss / total, accuracy=correct / total)

    # Print epoch statistics
    epoch_loss = running_loss / len(train_subset)
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2%}")

Epoch 1/5: 100%|██████████| 161/161 [01:05<00:00,  2.45batch/s, accuracy=0.67, loss=0.622] 


Epoch [1/5], Loss: 0.6223, Accuracy: 67.00%


Epoch 2/5: 100%|██████████| 161/161 [01:05<00:00,  2.47batch/s, accuracy=0.728, loss=0.518]


Epoch [2/5], Loss: 0.5184, Accuracy: 72.76%


Epoch 3/5: 100%|██████████| 161/161 [01:06<00:00,  2.42batch/s, accuracy=0.756, loss=0.481]


Epoch [3/5], Loss: 0.4810, Accuracy: 75.56%


Epoch 4/5: 100%|██████████| 161/161 [01:04<00:00,  2.50batch/s, accuracy=0.764, loss=0.488]


Epoch [4/5], Loss: 0.4884, Accuracy: 76.42%


Epoch 5/5: 100%|██████████| 161/161 [01:06<00:00,  2.41batch/s, accuracy=0.769, loss=0.496]

Epoch [5/5], Loss: 0.4964, Accuracy: 76.89%


In [136]:
# Set the model to evaluation mode
model.eval()

# Variables to keep track of accuracy and loss
val_loss = 0.0
val_correct = 0
val_total = 0

# Disable gradient computation for validation
with torch.no_grad():
    # Iterate over the validation set
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Compute loss
        val_loss += loss.item() * inputs.size(0)

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(val_dataset)
val_accuracy = val_correct / val_total

# Print validation results
print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")

Validation Loss: 0.4679, Validation Accuracy: 76.60%


<h2> More complex model WITH transformations such as horisontal flips, colorjitter etc

In [149]:
import torchvision
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms

# Define random transformations for data augmentation
random_transforms = transforms.RandomApply([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.5, contrast=0.25, saturation=0.2, hue=0.15)
], p=0.2)  # Apply each transformation with 30% probability

# Use the provided vggtransforms for preprocessing
transform = transforms.Compose([
    random_transforms,               # Apply random transformations for data augmentation
    transforms.Resize((224, 224)),   # Resize images to 224x224
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

#define train and valdataset again but with complex transformers
#need to redefine everything so the new transformers are applied correctly
train_dataset = ImageFolder(train_path, transform=transform)
val_dataset = ImageFolder(val_path, transform=transform)

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

subset_size = int(0.2 * len(train_dataset))

subset_indices = np.random.choice(len(train_dataset), subset_size, replace=False)

train_subset = Subset(train_dataset, subset_indices)

train_loader_subset = DataLoader(train_subset, batch_size=batch_size, shuffle=True)

In [150]:
#load in weights from the model
weights_id = torchvision.models.VGG16_Weights.IMAGENET1K_V1

vggmodel = models.vgg16(weights=weights_id)
vggmodel.eval()  # Set the model to evaluation mode

# Freeze the parameters of the model
for param in vggmodel.parameters():
    param.requires_grad = False

# Modify the classifier layer
num_features = vggmodel.classifier[6].in_features
vggmodel.classifier[6] = nn.Linear(num_features, len(train_dataset.classes))

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vggmodel = vggmodel.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vggmodel.parameters(), lr=0.001)

In [152]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    vggmodel.train()
    running_loss = 0.0
    correct = 0
    total = 0
    # Use tqdm for progress bar
    with tqdm(train_loader_subset, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as t:
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = vggmodel(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # Update tqdm progress bar
            t.set_postfix(loss=running_loss / total, accuracy=correct / total)

    epoch_loss = running_loss / len(train_subset)
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2%}")

Epoch 1/10:  25%|██▌       | 41/161 [01:07<03:19,  1.66s/batch, accuracy=0.698, loss=0.574]


KeyboardInterrupt: 

In [ ]:
# Set the model to evaluation mode
vggmodel.eval()

# Variables to keep track of accuracy and loss
val_loss = 0.0
val_correct = 0
val_total = 0

# Use tqdm for progress bar
with tqdm(val_loader, desc="Validation", unit="batch") as t:
    # Disable gradient computation for validation
    with torch.no_grad():
        # Iterate over the validation set
        for inputs, labels in t:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass
            outputs = vggmodel(inputs)
            loss = criterion(outputs, labels)

            # Compute loss
            val_loss += loss.item() * inputs.size(0)

            # Compute accuracy
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

            # Update tqdm progress bar
            t.set_postfix(loss=val_loss / val_total, accuracy=val_correct / val_total)

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(val_dataset)
val_accuracy = val_correct / val_total

# Print validation results
print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}")
